# AMA - Progetto Fabbisogni - Emesso Non Riscosso

# INIT

In [ ]:
%run Init.ipynb


Repo name: AMA_Progetto_Fabbisogni
Repo path: /home/bifani/GitHub/AMA_Progetto_Fabbisogni



<IPython.core.display.Javascript object>

In [ ]:
import datetime

import numpy as np
import pandas as pd

from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

from src.datafactory import DataFactory
from src.modelfactory import ModelFactory
from src.statsfactory import StatsFactory
from src.miscellaneous import *

from src.enr import *

<IPython.core.display.Javascript object>

# FUNCTIONS

In [ ]:
def importo_gb(df, cols, freq="M", bins=None, query=None, sort=False, count=False):
    grouper = list()
    for c in cols:
        if "DATA" in c:
            grouper.append(pd.Grouper(key=c, dropna=False, freq=freq))
        elif bins and "SUPERFICIE" in c:
            grouper.append(pd.cut(df[c], bins, right=False))
        else:
            grouper.append(pd.Grouper(key=c, dropna=False))

    df_ = df.groupby(grouper, dropna=False)

    func = {
        "IMPORTO_CONTRATTO": ["count", "sum"],
        "IMPORTO_PAREGGIO": ["sum"],
    }

    if "IMPORTO_PAGAMENTO" in df.columns:
        func["IMPORTO_PAGAMENTO"] = ["sum"]

    df_ = df_[list(func.keys())].agg(func=func)

    df_[("IMPORTO_DELTA", "sum")] = df_[("IMPORTO_CONTRATTO", "sum")] - df_[("IMPORTO_PAREGGIO", "sum")]
    df_[("IMPORTO_DELTA_PERC", "sum")] = df_[("IMPORTO_DELTA", "sum")] / df_[("IMPORTO_CONTRATTO", "sum")] * 100

    if sort:
        df_.sort_values([("IMPORTO_DELTA", "sum")], ignore_index=False, inplace=True)

    with pd.option_context("display.max_rows", None):
        if query and not count:
            print('df_[("IMPORTO_DELTA", "sum")] != 0')
            display(df_[df_[("IMPORTO_DELTA", "sum")] != 0])
        #             display(df_.query(query))
        else:
            display(df_)

<IPython.core.display.Javascript object>

# DATA

## SQLITE

In [ ]:
!grep -e "SQLITE_DB}" ../src/enr_db.py | sort

In [ ]:
DataFactory(f"{BASE_FOLDER_ENR}/{SQLITE_DB} | ", dry=True)

## DOM

### CONTRATTI_ATTIVI

In [ ]:
dom_ca = ENR_DOM_CONTRATTI_ATTIVI(
    # nrows=1000,
    #     dry=True,
    #     verbose=False,
    pre_process=True,
    process=True,
    post_process=True,
    silent=False,
)

In [ ]:
dom_ca.print_counts("CNT_TCN_COD")

### AVVISI_CONTRATTI_ATTIVI

In [ ]:
dom_aca = ENR_DOM_AVVISI_CONTRATTI_ATTIVI(
    # nrows=1000,
    pre_process=True,
    process=True,
    post_process=True,
    silent=False,
)

In [ ]:
dom_aca.df["IMPORTO_CONTRATTO"].sum()

In [ ]:
dom_aca.print_counts("MOTIVO_PAREGGIO")

### MERGE

In [ ]:
dom = ENR_DOM(pre_process=True, process=True, post_process=True, verbose=False, empty=True)

In [ ]:
dom.merge_CONTRATTI()

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_GE_VIE()

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_DECESSI()

In [ ]:
dom.print_counts("FLAG_DECEDUTO")

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_CESSAZIONI()

In [ ]:
dom.print_counts("FLAG_CESSATA")

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_RECAPITI()

In [ ]:
dom.print_counts("FLAG_RECAPITO")

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_ESENZIONI()

In [ ]:
dom.print_counts(["FLAG_ESENTE_SAP", "FLAG_ESENTE_RIC", "FLAG_ESENTE"])

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_RATEIZZAZIONI()

In [ ]:
dom.print_counts(["FLAG_RATE_SAP", "FLAG_RATE_RIC", "FLAG_RATE"])

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_BOLLETTINI()

In [ ]:
dom.print_counts("FLAG_BOLLETTINO")

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_COPIE_CONFORMI()

In [ ]:
dom.print_counts("FLAG_COPIA_CONFORME")

In [ ]:
dom.save("DOM")

In [ ]:
dom.merge_PAGAMENTI_F24()

In [ ]:
dom.print_counts("FLAG_F24")

In [ ]:
dom.df.query("FLAG_F24==1").value_counts("CNT_COD_FSC")

In [ ]:
dom.df.query("FLAG_F24==1 & CNT_COD_FSC=='04069160580'")

In [ ]:
func = {
    #     "CNT_TCN_COD": "first",
    #     "MOTIVO_PAREGGIO": "first",
    #     "CNT_COD": "first",
    #     "UTZ_CONTRATTO": "first",
    "IMPORTO_CONTRATTO": "sum",
    "IMPORTO_PAREGGIO": "sum",
    "IMPORTO_F24": "min",
}

for ctc in ["F", "G"]:
    df = dom.df.query(f"FLAG_F24==1 & MOTIVO_PAREGGIO.isnull() & CNT_TCN_COD=='{ctc}'").groupby("CNT_COD_FSC").agg(func)
    display(df)
    c = df["IMPORTO_F24"].sum()
    print(c)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD", "MOTIVO_PAREGGIO", "FLAG_F24"],
    #     query = "MOTIVO_PAREGGIO.isnull()"
)

In [ ]:
dom.save("DOM")

In [ ]:
dom.save(filename="DOM", ext="csv")

### SUMMARY

In [ ]:
dom.save("DOM_FINAL")
dom.save("DOM_FINAL", ext="csv")

In [ ]:
dom = ENR_DOM(
    pre_process=True,
    process=True,
    post_process=True,
    verbose=False,
)

In [ ]:
dom.print_counts(["CNT_COD", "UTZ_CONTRATTO"])

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD"],
)

In [ ]:
dom.print_importi(columns=["CNT_TCN_COD"], query="MOTIVO_PAREGGIO.notnull()")

In [ ]:
dom.print_importi(columns=["CNT_TCN_COD"], query="MOTIVO_PAREGGIO.isnull()")

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD", "MOTIVO_PAREGGIO"],
)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD", "TIPO_DOC_PAREGGIO"],
)

In [ ]:
dom.print_importi(columns=["CNT_TCN_COD", "TIPO_DOC_PAREGGIO"], query="MOTIVO_PAREGGIO.notnull()")

In [ ]:
dom.df["IMPORTO_CONTRATTO"].sum()

In [ ]:
dom.df.query("MOTIVO_PAREGGIO.isnull()")["IMPORTO_CONTRATTO"].sum()

In [ ]:
dom.df.query("MOTIVO_PAREGGIO.notnull()")["IMPORTO_CONTRATTO"].sum()

In [ ]:
dom.print_flags()

In [ ]:
dom.print_importi_flags(columns=["CNT_TCN_COD"], query="MOTIVO_PAREGGIO.isnull()")

In [ ]:
dom.print_importi_flags(
    columns=["MOTIVO_PAREGGIO"],
    query="MOTIVO_PAREGGIO.isnull()",
    #     flags=["FLAG_NON_RECAPITATO", "FLAG_DATI_ERRATI", "FLAG_RATE", "FLAG_ESENTE"],
)

In [ ]:
dom.print_crosstab(
    ["MOTIVO_PAREGGIO", "TIPO_DOC_PAREGGIO"],
)

In [ ]:
dom.print_crosstab(["MOTIVO_PAREGGIO", "TIPO_DOC_PAREGGIO"], query="IMPORTO_CONTRATTO<0")

In [ ]:
dom.print_crosstab()

In [ ]:
dom.print_crosstab(
    [
        "FLAG_BOLLETTINO",
        "FLAG_NON_RECAPITATO",
        "FLAG_DATI_ERRATI",
        "FLAG_RATE",
        "FLAG_ESENTE",
    ],
    query="(FLAG_NON_RECAPITATO==1|FLAG_DATI_ERRATI==1|FLAG_RATE==1|FLAG_ESENTE==1) & FLAG_BOLLETTINO==1",
    split=1,
)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD"],
    query="(FLAG_NON_RECAPITATO==1|FLAG_DATI_ERRATI==1|FLAG_RATE==1|FLAG_ESENTE==1) & FLAG_BOLLETTINO==1",
)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD"],
    query="(FLAG_NON_RECAPITATO==1|FLAG_DATI_ERRATI==1|FLAG_RATE==1|FLAG_ESENTE==1) & FLAG_COPIA_CONFORME==1",
)

In [ ]:
dom.print_importi(
    columns=["CNT_TCN_COD"],
    query="(FLAG_NON_RECAPITATO==1|FLAG_DATI_ERRATI==1|FLAG_RATE==1|FLAG_ESENTE==1) & (FLAG_BOLLETTINO==1 | FLAG_COPIA_CONFORME==1)",
)

In [ ]:
dom.groupby(
    ["CNT_TCN_COD", "MOTIVO_PAREGGIO"],
    ["SUPERFICIE", "COMPONENTI"],
    ["mean", "median", "min", "max", "std"],
)

In [ ]:
dom.print_counts("COMPONENTI")

#### MUNICIPI

In [ ]:
dom.df.boxplot(column="IMPORTO_CONTRATTO", by="NUM_CIR")

In [ ]:
sns.displot(
    data=dom.df.query("IMPORTO_CONTRATTO>0 & IMPORTO_CONTRATTO<10000"),
    x="IMPORTO_CONTRATTO",
    hue="NUM_CIR",
    kind="hist",
    bins=100,
    rug=False,
)

In [ ]:
dom.print_municipi()

In [ ]:
x = list()
for i, m in enumerate(dom.df["NUM_CIR"].unique()):
    d = dom.df.query(f"NUM_CIR == '{m}'")["IMPORTO_CONTRATTO"].to_list()
    x.append(d)
    if i == 1:
        break

len(x)

In [ ]:
stats.kruskal(*x)

In [ ]:
sp.posthoc_dunn(x, p_adjust="holm") < 0.05

#### DATA_PAREGGIO

In [ ]:
importo_gb(
    dom.df,
    cols=["CNT_TCN_COD", "MOTIVO_PAREGGIO", "DATA_PAREGGIO"],
    freq="M",
    query="IMPORTO_DELTA != 0",
)

#### UTZ_TARIFTYP

In [ ]:
dom.print_importi(columns=["CNT_TCN_COD", "MOTIVO_PAREGGIO", "UTZ_TARIFTYP"], query="MOTIVO_PAREGGIO.isnull()")

In [ ]:
func = {
    "CNT_COD": ["count", pd.Series.nunique],
    "UTZ_CONTRATTO": [pd.Series.nunique],
    "CNT_COD_FSC": [pd.Series.nunique],
    "IMPORTO_CONTRATTO": ["sum"],
    #     "IMPORTO_PAREGGIO": ["sum"],
}

dom.df.query("MOTIVO_PAREGGIO.isnull()").groupby("UTZ_TARIFTYP").agg(func)

#### RCP_SGL_NAZ

In [ ]:
importo_gb(
    dom.df.query("IMPORTO_CONTRATTO > 0"),
    cols=["CNT_TCN_COD", "MOTIVO_PAREGGIO", "RCP_SGL_NAZ"],
    query="IMPORTO_DELTA != 0",
)

## NDOM

### CONTRATTI_ATTIVI

In [ ]:
ndom_ca = ENR_NDOM_CONTRATTI_ATTIVI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ndom_ca.print_counts("CNT_TCN_COD")

### AVVISI_CONTRATTI_ATTIVI

In [ ]:
ndom_aca = ENR_NDOM_AVVISI_CONTRATTI_ATTIVI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ndom_aca.df["IMPORTO_CONTRATTO"].sum()

In [ ]:
ndom_aca.print_counts("MOTIVO_PAREGGIO")

### MERGE

In [ ]:
ndom = ENR_NDOM(pre_process=True, process=True, post_process=True, verbose=False, empty=True)

In [ ]:
ndom.merge_CONTRATTI()

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_GE_VIE()

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_DECESSI()

In [ ]:
ndom.print_counts("FLAG_DECEDUTO")

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_CESSAZIONI()

In [ ]:
ndom.print_counts(["FLAG_CESSATA", "RI_CLEAN_STA_PAR_IVA"])

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_PEC()

In [ ]:
ndom.print_counts(["FLAG_PEC_SAP", "FLAG_PEC_INV", "FLAG_NOPEC", "FLAG_PEC"])

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_GET()

In [ ]:
ndom.print_counts(["FLAG_GET_ATTIVITA"])

In [ ]:
ndom.print_counts(["FLAG_GET_PRATICHE"])

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_ESENZIONI()

In [ ]:
ndom.print_counts(["FLAG_ESENTE_SAP", "FLAG_ESENTE_RIC", "FLAG_ESENTE"])

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_RATEIZZAZIONI()

In [ ]:
ndom.print_counts(["FLAG_RATE_SAP", "FLAG_RATE_RIC", "FLAG_RATE"])

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_BOLLETTINI()

In [ ]:
ndom.print_counts("FLAG_BOLLETTINO")

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_COPIE_CONFORMI()

In [ ]:
ndom.print_counts("FLAG_COPIA_CONFORME")

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.merge_PAGAMENTI_F24()

In [ ]:
ndom.print_counts("FLAG_F24")

In [ ]:
func = {
    #     "CNT_TCN_COD": "first",
    #     "MOTIVO_PAREGGIO": "first",
    #     "CNT_COD": "first",
    #     "UTZ_CONTRATTO": "first",
    "IMPORTO_CONTRATTO": "sum",
    "IMPORTO_PAREGGIO": "sum",
    "IMPORTO_F24": "min",
}

for ctc in ["F", "G"]:
    df = (
        ndom.df.query(f"FLAG_F24==1 & MOTIVO_PAREGGIO.isnull() & CNT_TCN_COD=='{ctc}'").groupby("CNT_COD_FSC").agg(func)
    )
    display(df)
    c = df["IMPORTO_F24"].sum()
    print(df.shape, c)

In [ ]:
ndom.save("NDOM")

In [ ]:
ndom.save(filename="NDOM", ext="csv")

### SUMMARY

In [ ]:
ndom.save("NDOM_FINAL")
ndom.save("NDOM_FINAL", ext="csv")

In [ ]:
ndom = ENR_NDOM(
    pre_process=True,
    process=True,
    post_process=True,
    verbose=False,
)

In [ ]:
ndom.print_counts(["CNT_COD", "UTZ_CONTRATTO"])

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD"],
)

In [ ]:
ndom.print_importi(columns=["CNT_TCN_COD"], query="MOTIVO_PAREGGIO.notnull()")

In [ ]:
ndom.print_importi(columns=["CNT_TCN_COD"], query="MOTIVO_PAREGGIO.isnull()")

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD", "MOTIVO_PAREGGIO"],
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD", "TIPO_DOC_PAREGGIO"],
)

In [ ]:
ndom.print_importi(columns=["CNT_TCN_COD", "TIPO_DOC_PAREGGIO"], query="MOTIVO_PAREGGIO.notnull()")

In [ ]:
ndom.print_flags()

In [ ]:
ndom.print_importi_flags(columns=["CNT_TCN_COD"], query="MOTIVO_PAREGGIO.isnull()")

In [ ]:
ndom.print_importi_flags(
    columns=["MOTIVO_PAREGGIO"],
    query="MOTIVO_PAREGGIO.isnull()",
    #     flags=["FLAG_NON_RECAPITATO", "FLAG_DATI_ERRATI", "FLAG_RATE", "FLAG_ESENTE"],
)

In [ ]:
ndom.print_crosstab(
    ["MOTIVO_PAREGGIO", "TIPO_DOC_PAREGGIO"],
)

In [ ]:
ndom.print_crosstab(["MOTIVO_PAREGGIO", "TIPO_DOC_PAREGGIO"], query="IMPORTO_CONTRATTO<0")

In [ ]:
ndom.print_crosstab(
    ["FLAG_NOPEC", "FLAG_PEC", "FLAG_PEC_COMB", "FLAG_NOPEC_COMB"],
    query="MOTIVO_PAREGGIO.isnull()",
)

In [ ]:
ndom.print_crosstab(
    [
        "FLAG_BOLLETTINO",
        "FLAG_COPIA_CONFORME",
        #         "FLAG_NON_RECAPITATO",
        #         "FLAG_DATI_ERRATI",
        #         "FLAG_RATE",
        #         "FLAG_ESENTE",
    ],
    query="(FLAG_NON_RECAPITATO==1|FLAG_DATI_ERRATI==1|FLAG_RATE==1|FLAG_ESENTE==1)",
    split=1,
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD"],
    query="(FLAG_NON_RECAPITATO==1|FLAG_DATI_ERRATI==1|FLAG_RATE==1|FLAG_ESENTE==1) & FLAG_BOLLETTINO==1",
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD"],
    query="(FLAG_NON_RECAPITATO==1|FLAG_DATI_ERRATI==1|FLAG_RATE==1|FLAG_ESENTE==1) & FLAG_COPIA_CONFORME==1",
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD"],
    query="(FLAG_NON_RECAPITATO==1|FLAG_DATI_ERRATI==1|FLAG_RATE==1|FLAG_ESENTE==1) & (FLAG_BOLLETTINO==1 | FLAG_COPIA_CONFORME==1)",
)

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD", "FLAG_GET_ATTIVITA", "FLAG_GET_PRATICHE"],
    query="MOTIVO_PAREGGIO.isnull() & (FLAG_DATI_ERRATI==0)",
)

In [ ]:
ndom.groupby(
    ["CNT_TCN_COD", "MOTIVO_PAREGGIO"],
    ["SUPERFICIE"],
    ["mean", "median", "min", "max", "std"],
)

In [ ]:
ndom.groupby(["NUM_CIR"], ["SUPERFICIE"], ["mean", "median"])

#### MUNICIPI

In [ ]:
ndom.df.boxplot(column="IMPORTO_CONTRATTO", by="NUM_CIR")

In [ ]:
ndom.df.boxplot(column="IMPORTO_PAREGGIO", by="NUM_CIR")

In [ ]:
sns.displot(
    data=ndom.df.query("IMPORTO_CONTRATTO>0 & IMPORTO_CONTRATTO<10000"),
    x="IMPORTO_CONTRATTO",
    hue="NUM_CIR",
    kind="hist",
    bins=100,
    rug=False,
)

In [ ]:
ndom.print_municipi()

In [ ]:
x = list()
for i, m in enumerate(ndom.df["NUM_CIR"].unique()):
    d = ndom.df.query(f"NUM_CIR == '{m}'")["IMPORTO_CONTRATTO"].to_list()
    x.append(d)
    if i == 1:
        break

len(x)

In [ ]:
stats.kruskal(*x)

In [ ]:
sp.posthoc_dunn(x, p_adjust="holm") < 0.05

#### DATA_PAREGGIO

In [ ]:
ndom.df.query("DATA_PAREGGIO.isnull() & IMPORTO_PAREGGIO != 0")

In [ ]:
importo_gb(
    ndom.df.query("IMPORTO_CONTRATTO > 0"),
    cols=["CNT_TCN_COD", "MOTIVO_PAREGGIO", "DATA_PAREGGIO"],
    freq="M",
    query="IMPORTO_DELTA != 0",
)

#### UTZ_TARIFTYP

In [ ]:
ndom.print_importi(
    columns=["CNT_TCN_COD", "UTZ_TARIFTYP"],
    query="MOTIVO_PAREGGIO.isnull()",
)

In [ ]:
func = {
    #     "CNT_COD": ["count", pd.Series.nunique],
    "UTZ_CONTRATTO": [pd.Series.nunique],
    #     "CNT_COD_FSC": [pd.Series.nunique],
    "IMPORTO_CONTRATTO": ["sum"],
    "IMPORTO_PAREGGIO": ["sum"],
}

tt = ndom.df.query("FLAG_ESENTE==0 & FLAG_RATE==0").groupby("UTZ_TARIFTYP").agg(func)
tt

In [ ]:
tt.to_csv(f"{BASE_FOLDER_ENR}/UTZ_TARIFTYP.csv")

#### SUPERFICIE

In [ ]:
ndom.df["SUPERFICIE"].describe()

In [ ]:
pd.cut(ndom.df["SUPERFICIE"], 10)

In [ ]:
importo_gb(
    ndom.df.query("IMPORTO_CONTRATTO > 0"),  # .query("SUPERFICIE > 25000"),
    cols=["CNT_TCN_COD", "MOTIVO_PAREGGIO", "SUPERFICIE"],
    query="IMPORTO_DELTA != 0",
    sort=True,
    bins=10,
)

In [ ]:
ndom.query("SUPERFICIE > 50000 & MOTIVO_PAREGGIO.isnull()")

In [ ]:
ndom.query("FLAG_ESENTE == True & FLAG_RATE == True")

In [ ]:
ndom.pivot_table(
    ["FLAG_ESENTE", "FLAG_RATE"],
    "FLAG_PEC_COMB",
    "IMPORTO_CONTRATTO",
    ["sum", "mean"],
)

### FIT

In [ ]:
ndom = ENR_NDOM(
    pre_process=True,
    process=True,
    post_process=True,
    verbose=False,
)

In [ ]:
ndom.df = ndom.df[
    [
        "IMPORTO_CONTRATTO",
        "IMPORTO_PAREGGIO",
        "PERCENTUALE_NON_PAGATO",
        "FLAG_PAGATO",
        "MOTIVO_PAREGGIO",
        "TIPO_DOC_PAREGGIO",
        "CNT_TCN_COD",
        "SUPERFICIE",
        "CAP",
        "COD_ZON_URB",
        "NUM_CIR",
        "FLAG_NON_RECAPITATO",
        "FLAG_DATI_ERRATI",
        "FLAG_ESENTE",
        "FLAG_RATE",
    ]
]
ndom.update()

In [ ]:
ndom.save("ndom", ext="csv")

In [ ]:
ndom.plot_scatter("IMPORTO_CONTRATTO", "PERCENTUALE_NON_PAGATO")

In [ ]:
from statsmodels.graphics.gofplots import qqplot

# plt.figure()
fig = qqplot(ndom.df["PERCENTUALE_NON_PAGATO"], line="q")
fig.show()
plt.tight_layout()

In [ ]:
df = ndom.df[
    [
        "IMPORTO_CONTRATTO",
        "IMPORTO_PAREGGIO",
        "PERCENTUALE_NON_PAGATO",
        "FLAG_PAGATO",
        "MOTIVO_PAREGGIO",
        "TIPO_DOC_PAREGGIO",
        "CNT_TCN_COD",
        "SUPERFICIE",
        "CAP",
        "COD_ZON_URB",
        "NUM_CIR",
        "FLAG_NON_RECAPITATO",
        "FLAG_DATI_ERRATI",
        "FLAG_ESENTE",
        "FLAG_RATE",
    ]
].copy(deep=True)

ddf = DataFactory(df)

In [ ]:
ddf.encode_columns("CNT_TCN_COD", method="Ordinal")

In [ ]:
ddf.encode_columns("CNT_TCN_COD", method="OneHot")

In [ ]:
ddf.encode_columns("SUPERFICIE", method="Binarizer")

In [ ]:
ddf.encode_columns("SUPERFICIE", method="cut")

In [ ]:
ndom.df

In [ ]:
model = ModelFactory(ndom, label="FLAG_PAGATO")

In [ ]:
model.create_classifier("Logistic")

In [ ]:
model.fit(columns=["IMPORTO_CONTRATTO", "IMPORTO_PAREGGIO", "SUPERFICIE"])

## RECAPITI

In [ ]:
dom_rec = ENR_DOM_RECAPITI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
dom_rec.print_counts("FLAG_RECAPITO")

In [ ]:
dom_rec.print_counts("RCP_CMN_DES")

In [ ]:
dom_rec.df["RCP_CMN_DES"].str.isalpha().value_counts()

In [ ]:
dom_rec.query("RCP_CMN_DES == 'SUSTRI VT'")

In [ ]:
geo = ENR_COMUNI(pre_process=True, process=True, post_process=True, silent=True)

In [ ]:
"MONTERADO" in geo.comuni()

## PEC

In [ ]:
ndom_nopec = ENR_NDOM_NOPEC(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ndom_pec = ENR_NDOM_ESITI_PEC(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ndom_pec.print_counts("DESCRIZIONE_ERRORE_SEIPEC")

In [ ]:
ndom_pec.print_counts("SEMESTRE")

## GET

In [ ]:
get_att = ENR_GET_ATTIVITA(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
get_att.print_counts(["STATO"])

In [ ]:
get_prt = ENR_GET_PRATICHE(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
get_prt.print_counts(["STATO_LAVORAZIONE"])

## GE

In [ ]:
ge_t = ENR_GE_TERRIT(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ge_t.print_counts("COD_VIA")

In [ ]:
ge_t.print_counts("NUM_CIR")

In [ ]:
ge_t.print_duplicates(["COD_VIA"])

In [ ]:
ge_t.query("LIM_CIV_DA == LIM_CIV_A")

In [ ]:
ge_t.query("COD_VIA=='100045'")

In [ ]:
ge_t.query("COD_VIA == '1001'")

In [ ]:
ge_t.query("COD_VIA == '99835'")

In [ ]:
ge_v = ENR_GE_VIE(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ge_v.merge_GE_TERRIT()

In [ ]:
ge_v.print_duplicates()

In [ ]:
ge_v.print_counts("COD_VIA")

## DECESSI

In [ ]:
dec = ENR_DECESSI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
dec.print_counts("VERIFICA_SOGG")

## CESSAZIONI

In [ ]:
ces = ENR_CESSAZIONI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ces.print_counts(["RI_CLEAN_STA_PAR_IVA", "FLAG_CESSATA"])

## ESENZIONI

In [ ]:
ese_sap = ENR_ESENZIONI_SAP(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ese_r = ENR_ESENZIONI_RICHIESTE(
    pre_process=True,
    process=True,
    post_process=True,
)

## RATEIZZAZIONI

In [ ]:
rate_sap = ENR_RATEIZZAZIONI_SAP(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
rate_r = ENR_RATEIZZAZIONI_RICHIESTE(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
rate_r.print_counts("Stato")

## RECLAMI

In [ ]:
reclami = ENR_RECLAMI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
reclami.print_counts("STATO")

In [ ]:
reclami.groupby(["CATEGORIA", "CANALEDITRASMISSIONE", "RAGGRUPPAMENTO"], ["ID"], ["count"])

## PAGAMENTI

In [ ]:
pag = ENR_PAGAMENTI(
    pre_process=True,
    process=True,
    post_process=True,
)

### NDOM

In [ ]:
ndom = ENR_NDOM(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
ndom.merge_PAGAMENTI()

In [ ]:
ndom.df["IMPORTO_PAREGGIO"].sum()

In [ ]:
ndom.df["IMPORTO_PAGAMENTO"].sum()

In [ ]:
ndom.query("IMPORTO_PAREGGIO != IMPORTO_PAGAMENTO & IMPORTO_PAGAMENTO != 0")

In [ ]:
ndom2.df["PAREGGIO"] = ~ndom2.df["DATA_PAREGGIO"].isna()
ndom2.df["PAGAMENTO"] = ~ndom2.df["DATA_PAGAMENTO"].isna()

ndom2.print_counts("PAREGGIO")
ndom2.print_counts("PAGAMENTO")

In [ ]:
ndom2.df[(ndom2.df["PAGAMENTO"] == True) & (ndom2.df["PAREGGIO"] == False)]

In [ ]:
ndom2.df[(ndom2.df["PAGAMENTO"] == True) & (ndom2.df["IMPORTO_PAREGGIO"] == 0)]

In [ ]:
(ndom2.df[(ndom2.df["PAGAMENTO"] == True) & (ndom2.df["IMPORTO_PAREGGIO"] == 0)]).value_counts(
    "MOTIVO_PAREGGIO", dropna=False
)

## BONIFICI

In [ ]:
bnf = ENR_PAGAMENTI_BONIFICI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
bnf.groupby(["CCP"], ["IMPORTO_BON"], ["sum"])

In [ ]:
bnf_ = bnf.df.query(
    "NOTE.str.contains('TA[., ]?RI',regex=True,case=False) & NOTE.str.contains('sem',regex=True,case=False)"
)
bnf_ = bnf_.query(
    "~NOTE.str.contains('II',regex=True,case=False) & ~NOTE.str.contains('2o',regex=True,case=False) & ~NOTE.str.contains('[\s]sec',regex=True,case=False)"
)
display(bnf_)
bnf_.groupby("CCP", dropna=False)["IMPORTO_BON"].agg("sum").to_frame()

In [ ]:
note = bnf_["NOTE"]
with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
    display(note.values)

## F24

In [ ]:
f24 = ENR_PAGAMENTI_F24(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
f24.print_counts("Rateaz")

In [ ]:
sorted(f24.df.value_counts("Rateaz").index)

In [ ]:
f24.groupby(["CodTrib"], ["IMPORTO_F24"], ["sum"])

In [ ]:
f24.query("CodFis=='04069160580'")

## PAGO PA

In [ ]:
rp = ENR_PAGAMENTI_PAGOPA(
    nrows=10000,
    #     dry=True,
    #     verbose=False,
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
rp.groupby(
    ["RETEINCASSO"],
    ["IMPORTO_AVVISO", "IMPORTO_PAGOPA", "IMPORTOTOTALEACCREDITATO"],
    ["sum"],
)

## BOLLETTINI

In [ ]:
bol = ENR_BOLLETTINI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
bol.print_duplicates("keys")

## COPIE CONFORMI

In [ ]:
cc = ENR_COPIE_CONFORMI(
    pre_process=True,
    process=True,
    post_process=True,
)

In [ ]:
bol.print_duplicates("keys")